In [ ]:
''' No need to run
! conda intall -c anaconda tensorflow -y
! conda install -c conda-forge gym -y
! pip show gym
'''

In [ ]:
import sys
sys.path.append('/home/kimnj3050/.conda/envs/njenv/lib/python3.7/site-packages')
from gym import Env #customer env
from gym.spaces import Discrete, Box, Tuple, Dict, MultiBinary, MultiDiscrete  #can define action, current state about env
import numpy as np
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
class CounterEnv_DQN(Env):
    def __init__(self):
        #actions increment, decrement, consistent
        self.action_space = Discrete(3)
        self.observation_space = Box(low=np.array([0]), high = np.array([10]))
        #set start number
        self.state = 0 + random.randint(-2,2)
        #set counter length
        self.counter_length = 10
        
    def step(self, action):
        #apply action
        #0 -1 = -1 -> decrement
        #1 -1 = 0 -> consistent
        #2 -1 = 1 -> increment
        self.state += action -1
        #reduce counter length by 1 time
        self.counter_length -= 1
        
        #calculate reward
        if self.state >= 8:
            reward = 1
        else:
            reward = -1
            
        #check if counter is done
        if self.counter_length <= 0:
            done = True
        else: 
            done = False
        
        #noise
        self.state += random.randint(-1,1)
        
        info = {}
        
        #return step information
        return self.state, reward, done, info
    
    def render(self):
        pass
    
    def reset(self):
        #reset counter
        self.state = 0 + random.randint(-2,2)
        #reset counter time
        self.counter_length = 10
        return self.state

In [ ]:
env = CounterEnv_DQN()

In [ ]:
env.observation_space.sample()

In [ ]:
env.action_space.sample()

In [ ]:
episodes = 20
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{} reward:{} '.format(episode, score, reward))
    


In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
states = env.observation_space.shape
actions = env.action_space.n

In [ ]:
def build_model(states, actions):
    model = Sequential()
    model.add(Dense(24, activation = 'relu', input_shape = states))
    model.add(Dense(24, activation = 'relu'))
    model.add(Dense(action, activation ='linear'))
    return model

In [ ]:
model = build_model(states, actions)

In [ ]:
model.summary()

In [ ]:
from rl.agents import DQNAgent
from rl.plolicy import BoltzmannQPolicy
from rl.memory import SequentialMemory